<center><img src="https://github.com/DACSS-CSSmeths/guidelines/blob/main/pics/small_logo_ccs_meths.jpg?raw=true" width="700"/></center>

_____
<a id='home'></a>


# Introduction to Optimization for Decision Making


# Part 2: Multicriteria Decision-Making

In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR_rRFAM3dbRH8R_Hbme-LmSGUxL_tDmy7BiHCUvtc_2r_lvwO8cFrjMVYXbrV6Nw5XbkE3X2iLMXui/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

## AHP in Python

# The Comparissons


1. Prepare data file with the comparissons: You will use an excel file for this. In this step, you and the other decision makers will make pairwise comparissons for:
   
a. The alternatives for each criterion.

b. The criteria themselves.

I have prepared a GoogleSheets file, which looks like this:

In [2]:
%%html
<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQ5AXaOjU61DouEw_C1pFo6gyxTd_UCNGBHoX0JI6D4X9xt_aHKCTnVX-WtxLt8z2iW8-GePQTqFVXK/pubhtml?widget=true&amp;headers=false" width="600" height="300"></iframe>


Notice each sheet is a criterion, and the label of the sheet is a criterion written in **lower case**; then, the criteria sheet must use lower case in the table labels. It needs not be lower case, but whatever case you use has to be same accross all labels.

2. Get the data into Python

Let me get the link to that GoogleSheet file as an Excel file:

In [3]:
# the link to the data

linkGoogle='https://docs.google.com/spreadsheets/d/e/2PACX-1vQ5AXaOjU61DouEw_C1pFo6gyxTd_UCNGBHoX0JI6D4X9xt_aHKCTnVX-WtxLt8z2iW8-GePQTqFVXK/pub?output=xlsx'# the link to the data

3. Open each sheet:

Above you have the link, but not the data yet. Let me get each comparisson sheet:

In [4]:
# this one first
import pandas as pd

pairwise_age=pd.read_excel(linkGoogle,sheet_name='age')
pairwise_age

,Unnamed: 0,Tom,Dick,Harry
0,Tom,NaN,NaN,5.0
1,Dick,3.0,NaN,9.0
2,Harry,NaN,NaN,NaN


That table will **not** work well, we need this one:

In [5]:
pairwise_age=pd.read_excel(linkGoogle,sheet_name='age', index_col=0) # notice
pairwise_age

,Tom,Dick,Harry
Tom,NaN,NaN,5.0
Dick,3.0,NaN,9.0
Harry,NaN,NaN,NaN


Now you know how to read the rest:

In [6]:
# opening the comparissons
pairwise_experience=pd.read_excel(linkGoogle,sheet_name='experience', index_col=0)
pairwise_education=pd.read_excel(linkGoogle,sheet_name='education', index_col=0)
pairwise_charisma=pd.read_excel(linkGoogle,sheet_name='charisma', index_col=0)
pairwise_criteria=pd.read_excel(linkGoogle,sheet_name='criteria', index_col=0)

Check the criteria, just in case:

In [7]:
pairwise_criteria

,experience,education,charisma,age
experience,NaN,4.0,3.0,7.0
education,NaN,NaN,NaN,3.0
charisma,NaN,3.0,NaN,5.0
age,NaN,NaN,NaN,NaN


All the comparissons we have are in a format known as **adjacency matrix**. We saw that in the network session, so now we can use some of that.

# The Data Preparation

4. Transform all adjacency matrices into pairwise comparissons:

This is a just a code to make the comparissons readable in the AHP library we will use. 

In [8]:
import networkx as nx

G_age = nx.from_pandas_adjacency(pairwise_age, # just change this pairwase
                                 create_using=nx.MultiDiGraph())

# pairwise
G_age.edges(data=True)

OutMultiEdgeDataView([('Tom', 'Tom', {'weight': nan}), ('Tom', 'Dick', {'weight': nan}), ('Tom', 'Harry', {'weight': 5.0}), ('Dick', 'Tom', {'weight': 3.0}), ('Dick', 'Dick', {'weight': nan}), ('Dick', 'Harry', {'weight': 9.0}), ('Harry', 'Tom', {'weight': nan}), ('Harry', 'Dick', {'weight': nan}), ('Harry', 'Harry', {'weight': nan})])

We do not need the missing values (nan), then what about:

In [9]:
import numpy as np 
{(e[0],e[1]):e[2]['weight'] for e in G_age.edges(data=True) if np.isfinite(e[2]['weight'])}

{('Tom', 'Harry'): 5.0, ('Dick', 'Tom'): 3.0, ('Dick', 'Harry'): 9.0}

We wil use that code to get all the comparissons:

In [10]:

age_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_age.edges(data=True) if np.isfinite(e[2]['weight'])}

# ... and the remaining comparissons:
G_exp = nx.from_pandas_adjacency(pairwise_experience,create_using=nx.MultiDiGraph())
experience_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_exp.edges(data=True) if np.isfinite(e[2]['weight'])}

G_edu = nx.from_pandas_adjacency(pairwise_education,create_using=nx.MultiDiGraph())
education_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_edu.edges(data=True) if np.isfinite(e[2]['weight'])}

G_cha = nx.from_pandas_adjacency(pairwise_charisma,create_using=nx.MultiDiGraph())
charisma_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_cha.edges(data=True) if np.isfinite(e[2]['weight'])}

In [11]:
# take a look
[age_comparisons, experience_comparisons,education_comparisons,charisma_comparisons]

[{('Tom', 'Harry'): 5.0, ('Dick', 'Tom'): 3.0, ('Dick', 'Harry'): 9.0},
 {('Tom', 'Harry'): 4.0, ('Dick', 'Tom'): 4.0, ('Dick', 'Harry'): 9.0},
 {('Tom', 'Dick'): 3.0, ('Harry', 'Tom'): 5.0, ('Harry', 'Dick'): 7.0},
 {('Tom', 'Dick'): 5.0, ('Tom', 'Harry'): 9.0, ('Dick', 'Harry'): 4.0}]

In [12]:
# now the criteria

G_CRIT = nx.from_pandas_adjacency(pairwise_criteria,create_using=nx.MultiDiGraph())
criteria_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_CRIT.edges(data=True) if np.isfinite(e[2]['weight'])}
criteria_comparisons

{('experience', 'education'): 4.0,
 ('experience', 'charisma'): 3.0,
 ('experience', 'age'): 7.0,
 ('education', 'age'): 3.0,
 ('charisma', 'education'): 3.0,
 ('charisma', 'age'): 5.0}

# AHP



5. The installation: we will install ahpy, but we also need a numpy version lower than 2.0

In [13]:
## install
# !pip install --force-reinstall numpy==1.26.4
# !pip install ahpy

Once installed, we can call the library and use the **Compare** function:

In [14]:
# input each comparisson

import ahpy

experience = ahpy.Compare('experience', experience_comparisons, random_index='saaty')
education = ahpy.Compare('education', education_comparisons, random_index='saaty')
charisma = ahpy.Compare('charisma', charisma_comparisons, random_index='saaty')
age = ahpy.Compare('age', age_comparisons, random_index='saaty')
criteria = ahpy.Compare('criteria', criteria_comparisons, random_index='saaty')

6. Create hierarchy:

Remember we have the **hierarchy** Goal <- Criteria <- Alternatives. At this stage, you just need to tell which are the children of the criteria:

In [15]:
criteria.add_children([experience, education, charisma, age])

We can see which criterion was more valuable like this:

In [16]:
print(criteria.global_weights)

{'experience': 0.5476, 'charisma': 0.2699, 'education': 0.1266, 'age': 0.0559}


7. See result:

Now, you may know which is the best option:

In [17]:
print(criteria.target_weights)

{'Dick': 0.4928, 'Tom': 0.3581, 'Harry': 0.1491}


8. Assess consistency

The AHP algorithm assumes that when you are comparing you are consistent; but it may detect if you have been inconsistent:

In [18]:
[(val.name,val.consistency_ratio) for val in [experience, education, charisma, age, criteria]]

[('experience', 0.0355),
 ('education', 0.0624),
 ('charisma', 0.0685),
 ('age', 0.0279),
 ('criteria', 0.0444)]

We should review comparissons if you get values greater than 0.1 (so, in this case the solution holds).

## Homework #3 (option B)

For the homework you need to watch this example:

In [19]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/ChkBNabdfjo?si=Y_r-ERhUKtvNd3Oj" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>


Go to Canvas for the instructions!